In [ ]:
# Simple Gen AI App

# RAG

In [1]:
# loading env variables

import os
from dotenv import load_dotenv

load_dotenv()

openai_key = os.getenv('OPENAI_API_KEY')
langsmith_key = os.getenv('LANGSMITH_API_KEY')
project = os.getenv('PROJECT_NAME')

os.environ['LANGSMITH_TRACING'] = "true"

In [5]:
# data ingestion

from langchain_community.document_loaders import WebBaseLoader

url = "https://docs.langchain.com/langsmith/administration-overview"

loader = WebBaseLoader(url)

document = loader.load()

type(document[0])

langchain_core.documents.base.Document

In [10]:

# splitting documents in to chunks

from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

docs = splitter.split_documents(document)

type(docs[0])

langchain_core.documents.base.Document

In [ ]:
# embedding : text to vectors

from langchain_openai import  OpenAIEmbeddings

embeder = OpenAIEmbeddings(model='text-embedding-3-large')

In [ ]:
# FAISS vectorstoresDB

from langchain_community.vectorstores import FAISS

db = FAISS.from_documents(docs,embeder)

In [ ]:
# testing

query = "Why auto-upgrade traces?"

result = db.similarity_search(query)

result[0]

In [ ]:
# llm model

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-5')

In [ ]:
# creating prompt template

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """Answer the following questions based on the provided context
    <context>
    {context}
    </context>
    Question : {input}"""
)

In [ ]:
# document chain

from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm,prompt)

document_chain

In [ ]:
# creating retriever 

retriever = db.as_retriever()

In [ ]:
# creating retrieval chain

from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(retriever,document_chain)

In [ ]:
response = retrieval_chain.invoke(
    {"input":"question from site?"}
)

response['answer']

In [ ]:
response